In [ ]:
# @title
!pip install pyspark

In [65]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format
from pyspark.ml.feature import StringIndexer, MinMaxScaler, PCA, VectorAssembler
from pyspark.ml.regression import LinearRegression

In [47]:
spark = SparkSession.builder.getOrCreate()

In [48]:
df_video = spark.read.parquet('/content/videos-comments-tratados.snappy.parquet')

In [55]:
df_video.show(5)

+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+
|   Video ID|               Title|Published At|Keyword|Likes|Comments| Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|Keyword Index|
+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Let's not forget ...|        1|           95|    8|         17.0|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Here in NZ 50% of...|        0|           19|    8|         17.0|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|I will forever ac...|        2|          161|    8|         17.0|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022

In [50]:
df_video = df_video.withColumn('Month', date_format(col('Published At'), 'MM'))
df_video.show(5)

+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+
|   Video ID|               Title|Published At|Keyword|Likes|Comments| Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|
+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Let's not forget ...|        1|           95|   08|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Here in NZ 50% of...|        0|           19|   08|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|I will forever ac...|        2|          161|   08|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Whenever I go to ...|        0

In [51]:
indexador = StringIndexer(inputCol='Keyword', outputCol='Keyword Index')
df_video = indexador.fit(df_video).transform(df_video)
df_video.show(5)

+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+
|   Video ID|               Title|Published At|Keyword|Likes|Comments| Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|Keyword Index|
+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Let's not forget ...|        1|           95|   08|         17.0|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Here in NZ 50% of...|        0|           19|   08|         17.0|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|I will forever ac...|        2|          161|   08|         17.0|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022

In [52]:
df_video.printSchema()

root
 |-- Video ID: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interaction: integer (nullable = true)
 |-- Year: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Sentiment: integer (nullable = true)
 |-- Likes Comment: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- Keyword Index: double (nullable = false)



In [53]:
df_video = df_video.withColumn('Year', col('Year').cast('int'))
df_video.printSchema()

root
 |-- Video ID: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interaction: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Sentiment: integer (nullable = true)
 |-- Likes Comment: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- Keyword Index: double (nullable = false)



In [54]:
df_video = df_video.withColumn('Month', col('Month').cast('int'))
df_video.printSchema()

root
 |-- Video ID: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interaction: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Sentiment: integer (nullable = true)
 |-- Likes Comment: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Keyword Index: double (nullable = false)



In [56]:
montar_vetor = VectorAssembler(inputCols=['Likes', 'Views', 'Year', 'Month', 'Keyword Index'], outputCol='features')
df_video = montar_vetor.transform(df_video)
df_video.show(5)

+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+
|   Video ID|               Title|Published At|Keyword|Likes|Comments| Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|Keyword Index|            features|
+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Let's not forget ...|        1|           95|    8|         17.0|[3407.0,135612.0,...|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Here in NZ 50% of...|        0|           19|    8|         17.0|[3407.0,135612.0,...|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|I

In [59]:
scaler = MinMaxScaler(inputCol='features', outputCol='Features Normal')
modelo_scaler = scaler.fit(df_video)
df_video = modelo_scaler.transform(df_video)
df_video.show(5)

+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+--------------------+
|   Video ID|               Title|Published At|Keyword|Likes|Comments| Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|Keyword Index|            features|     Features Normal|
+-----------+--------------------+------------+-------+-----+--------+------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+--------------------+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Let's not forget ...|        1|           95|    8|         17.0|[3407.0,135612.0,...|[2.07229197864298...|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672|135612|     139691|2022|Here in NZ 50% of...|        0|           19|    8|         17.0|[3407.0,135612.0,...|[2.072291978

In [62]:
pca = PCA(k=1, inputCol='Features Normal', outputCol='Features PCA')
modelo_pca = pca.fit(df_video)
df_video = modelo_pca.transform(df_video)
df_video.show()

+-----------+--------------------+------------+-------+-----+--------+-------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+--------------------+--------------------+
|   Video ID|               Title|Published At|Keyword|Likes|Comments|  Views|Interaction|Year|             Comment|Sentiment|Likes Comment|Month|Keyword Index|            features|     Features Normal|        Features PCA|
+-----------+--------------------+------------+-------+-----+--------+-------+-----------+----+--------------------+---------+-------------+-----+-------------+--------------------+--------------------+--------------------+
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|Let's not forget ...|        1|           95|    8|         17.0|[3407.0,135612.0,...|[2.07229197864298...|[0.4548135019714761]|
|wAZZ-UWGVHI|Apple Pay Is Kill...|  2022-08-23|   tech| 3407|     672| 135612|     139691|2022|Here in N

In [64]:
train_data, test_data = df_video.randomSplit([0.8, 0.2], seed=42)

In [67]:
regressao_linear = LinearRegression(featuresCol='Features Normal', labelCol='Comments')
modelo_regressao = regressao_linear.fit(train_data)

avaliar_test = modelo_regressao.evaluate(test_data)
print("RMSE: ", avaliar_test.rootMeanSquaredError)
print("R2: ", avaliar_test.r2)

RMSE:  25370.3336201662
R2:  0.6602413154888491


In [69]:
df_video.write.mode('overwrite').parquet('/content/videos-preparados-pca.snappy.parquet')